The idea is to use ORB descriptors to try to improve classification

In [69]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from skimage.feature import match_descriptors, ORB

from cat_dogs_functions import train_test_model, test_models, load_preprocess_dataset

In [70]:
with open("data_path.txt", "r") as f:
    data_path = f.read()

In [71]:
descriptor_extractor = ORB(n_keypoints=200)

In [72]:
images, labels = load_preprocess_dataset(data_path, 100, True, (128,128), False)

In [73]:
descriptors = []
for img in tqdm(images):
    descriptor_extractor.detect_and_extract(img)
    descriptors.append(descriptor_extractor.descriptors)

  0%|          | 0/200 [00:00<?, ?it/s]

In [74]:
# Import the models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [75]:
X_train, X_test, y_train, y_test = train_test_split(
    np.array(descriptors).reshape(len(descriptors),-1), 
    labels, 
    test_size=0.3, 
    shuffle=True,
    random_state=42,
    )

/tmp/ipykernel_24027/2891250366.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(descriptors).reshape(len(descriptors),-1),


In [76]:
models = {
    "knn": KNeighborsClassifier(),
    "logreg": LogisticRegression(),
    "svc": SVC(),
    "RFC": RandomForestClassifier(),
    # "GBC": GradientBoostingClassifier(),
    # "MLP": MLPClassifier()
}

In [77]:
tested_models = []
training_times = []
scores = []
deviations = []
# Train and test the models.
for name, model in models.items():
    tested_models.append(name)

    # Train and test
    model, model_accuracy, training_time, deviation = train_test_model(model, images, labels)

    # Track success
    scores.append(model_accuracy)
    deviations.append(deviation)
    training_times.append(training_time)

results = pd.DataFrame({
    "model": tested_models,
    "Accuracy": scores,
    "Stdev": deviations,
    "Training time": training_times
})

/home/quillaur/streamlit_apps/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/quillaur/streamlit_apps/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

In [78]:
results.sort_values(by="Accuracy", ascending=False, axis=0, inplace=True)
results

,model,Accuracy,Stdev,Training time
3,RFC,0.516667,0.753576,0.069722
2,svc,0.500000,0.683896,0.102062
0,knn,0.433333,0.001893,0.238630
1,logreg,0.433333,1.315215,0.170783


# References

* https://scikit-learn.org/stable/supervised_learning.html#supervised-learning
* https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_orb.html#sphx-glr-auto-examples-features-detection-plot-orb-py
* https://www.cours-gratuit.com/tutoriel-python/tutoriel-python-les-bases-de-traitement-dimages-avec-scikit-image